# Dimsum Concepts and Examples

Dimsum is a library for performing computations on labeled data.

This notebook walks through the core concepts of dimsum and shows some examples of usage.

## Concept 1: Dimension, Schema, CodedArray

The core object is a CodedArray which can be thought of as a pandas.Series with a MultiIndex of **unique** codes.

Prior to creating any CodedArray, a Schema of Dimensions must be defined. This fixed schema gives every possible combination of dimensions a unique 60-bit code, sort of like a big hash table.

In [1]:
import numpy as np
import pandas as pd
import dimsum as ds
from dimsum import Schema, Dimension, CalendarDimension

In [2]:
size = Dimension('size', ['S', 'M', 'L', 'XL'])
shape = Dimension('shape', ['Triangle', 'Rectangle', 'Circle'])
quality = Dimension('quality', ['Bad', 'Okay', 'Good'])
days = CalendarDimension('days', pd.period_range('2020-03-15', periods=180))
quarters = CalendarDimension('quarters', pd.period_range('2020-Q1', '2020-Q4', freq='Q'))
stock_ticker = Dimension('ticker', ['AAPL', 'MSFT'])

schema = Schema([size, shape, quality, days, quarters, stock_ticker])

Looking at the schema shows which bits (and how many) are reserved for each dimension.

In [3]:
schema

Schema:
  11100000000000000000 size
  00011000000000000000 shape
  00000110000000000000 quality
  00000001111111100000 days
  00000000000000011100 quarters
  00000000000000000011 ticker

In [92]:
schema.total_bits

20

Encoding data converts labels into codes, but hides this fact in the default display.

In [4]:
widget_cost = schema.load(
    [['S', 'Triangle', 4.50],
     ['S', 'Rectangle', 5.05],
     ['S', 'Circle', 7.00],
     ['M', 'Triangle', 11.10],
     ['M', 'Rectangle', 11.80],
     ['M', 'Circle', 14.00],
     ['L', 'Rectangle', 19.25],
     ['L', 'Circle', 25.00],
     ['XL', 'Circle', 36.00],
    ], ['size', 'shape']
)
widget_cost

,size,shape,* values *
0,S,Triangle,4.50
1,S,Rectangle,5.05
2,S,Circle,7.00
3,M,Triangle,11.10
4,M,Rectangle,11.80
5,M,Circle,14.00
6,L,Rectangle,19.25
7,L,Circle,25.00
8,XL,Circle,36.00


Viewing the codes is possible, and this will be useful in later analysis to have some understanding of the underlying codes.

In [5]:
widget_cost.codes()

,size,shape,* values *
0,S,Triangle,163840
1,S,Rectangle,196608
2,S,Circle,229376
3,M,Triangle,294912
4,M,Rectangle,327680
5,M,Circle,360448
6,L,Rectangle,458752
7,L,Circle,491520
8,XL,Circle,622592


To further illustrate this concept, let's look at an example of converting a set of labels into a code and back.

In [6]:
schema.encode(size='L', shape='Rectangle')

458752

In [7]:
schema.decode(458752)

{'size': 'L',
 'shape': 'Rectangle',
 'quality': '∅',
 'days': '∅',
 'quarters': '∅',
 'ticker': '∅'}

The number is masked with each dimension's bit pattern, then shifted to find the dimension code.

Notice that every dimension is represented. A code which has no "quality" value actually has the NULL (∅) quality value.

The NULL value is automatically added to each Dimension in position 0.

In [8]:
schema.dimension_enums('size')

,size,* values *
0,∅,0
1,S,1
2,M,2
3,L,3
4,XL,4


## Concept 2: Pivoted Data

A CodedArray can be shown in flat form or pivoted. The meaning doesn't change, but can be nicer to look at and is more intuitive
for things like reduction along several dimensions.

The pivoted form also shows the missing values more easily.

In [9]:
pivoted_cost = widget_cost.pivot(left='shape')
pivoted_cost

size,S,M,L,XL
shape,,,,
Triangle,4.5,11.1,,
Rectangle,5.05,11.8,19.25,
Circle,7,14,25,36


Dimsum seamlessly handles both flat and pivoted data in calculations.

Here we subtract the widget cost from its pivoted form. Dimsum aligns them and subtracts the correct values, so the results are all zeros.

In [10]:
widget_cost - pivoted_cost

,size,shape,* values *
0,S,Triangle,0.0
1,S,Rectangle,0.0
2,S,Circle,0.0
3,M,Triangle,0.0
4,M,Rectangle,0.0
5,M,Circle,0.0
6,L,Rectangle,0.0
7,L,Circle,0.0
8,XL,Circle,0.0


## Concept 3: Alignment

With a well defined schema, alignment happens automatically.

There are 4 cases to consider when aligning two CodedArrays:
1. Matching dimensions
2. Subset dimensions
3. Disjoint dimensions
4. Partially disjoint dimensions

### Matching dimensions perform an inner join

This means that missing values propagate, unless specified (see Expansion alignment below).

In [11]:
widget_sales = schema.load(
    [['S', 'Rectangle', 1],
     ['M', 'Circle', 4],
     ['L', 'Circle', 3]
    ], ['size', 'shape']
)
widget_sales

,size,shape,* values *
0,S,Rectangle,1
1,M,Circle,4
2,L,Circle,3


In this example, both widget_cost and widget_sales have dimensions {'size', 'shape'}.

While widget_cost has data for many combinations of size and shape, widget_sales only has data for three combinations. As a result, the product will only have those three matching rows.

In [12]:
widget_revenue = widget_sales * widget_cost
widget_revenue

,size,shape,* values *
0,S,Rectangle,5.05
1,M,Circle,56.00
2,L,Circle,75.00


### Subset dimensions are broadcast

If one of the objects has dimensions fully contained within the other object's dimensions, the data from the smaller object is broadcast.

In this example, the shipping cost only depends on the size, not the shape. The total cost for a Small Triangle is the widget cost plus the cost of shipping a small object.

In [13]:
shipping_cost = schema.load({'S': 1.00, 'M': 1.99, 'L': 2.25, 'XL': 5.00}, 'size')
shipping_cost

,size,* values *
0,S,1.00
1,M,1.99
2,L,2.25
3,XL,5.00


In [14]:
widget_cost + shipping_cost

shape,Triangle,Rectangle,Circle
size,,,
S,5.5,6.05,8
M,13.09,13.79,15.99
L,,21.5,27.25
XL,,,41


The shape of the combined cost is pivoted with the overlapping dimension(s) shown on the left.

### Disjoint dimensions perform an outer join

In [15]:
x = schema.load({'S': 1, 'M': 2, 'L': 3, 'XL': 4}, 'size')
y = schema.load({'Bad': -1, 'Okay': 1, 'Good': 5}, 'quality')

In [16]:
x * y

quality,Bad,Okay,Good
size,,,
S,-1,1,5
M,-2,2,10
L,-3,3,15
XL,-4,4,20


### Partially overlapping dimensions perform a hybrid

Overlapping dimensions match based on an inner join. In the following example, size=S is not defined in widget_premium, so it doesn't show up in the result.

Mismatched dimensions perform an outer join. In the example, shape and quality are disjoint dimensisons, so they perform an outer join **wherever the size matches**.

In [17]:
widget_premium = schema.load(
    [['M', 'Bad', -0.3], ['M', 'Good', 0.3],
     ['L', 'Bad', -0.4], ['L', 'Okay', 0.05], ['L', 'Good', 0.35],
     ['XL', 'Bad', -0.6], ['XL', 'Okay', 0.1], ['XL', 'Good', 0.4],
    ], ['size', 'quality']
)
widget_premium.pivot(top='quality')

quality,Bad,Okay,Good
size,,,
M,-0.3,,0.3
L,-0.4,0.05,0.35
XL,-0.6,0.1,0.4


In [18]:
widget_cost.pivot(left='size')

shape,Triangle,Rectangle,Circle
size,,,
S,4.5,5.05,7
M,11.1,11.8,14
L,,19.25,25
XL,,,36


In [19]:
quality_adjustment = widget_cost * widget_premium
quality_adjustment

shape,Triangle,Triangle,Rectangle,Rectangle,Rectangle,Circle,Circle,Circle
quality,Bad,Good,Bad,Okay,Good,Bad,Okay,Good
size,,,,,,,,
M,-3.33,3.33,-3.54,,3.54,-4.2,,4.2
L,,,-7.7,0.9625,6.7375,-10,1.25,8.75
XL,,,,,,-21.6,3.6,14.4


## Concept 4: Expansion during alignment

A simple expansion notation allows for left-join, right-join, and outer-join behavior for aligning matching dimensions.

In [20]:
x1 = schema.load({'S': 1, 'M': 2}, 'size')
x2 = schema.load({'S': 10, 'L': 50}, 'size')

By default, adding x1 and x2 will perform an inner join on the size dimension.

In [21]:
x1 + x2

,size,* values *
0,S,11


The notation `x1.X` indicates to expand `x1` to the same dimensions as the other object, filling its values with 0.

This creates a right-join.

In [22]:
x1.X + x2

,size,* values *
0,S,11
1,L,50


The default fill value of 0 can be modified using square brackets

In [23]:
x1.X[999] + x2

,size,* values *
0,S,11
1,L,1049


Expanding both objects will perform an outer join on the codes

In [24]:
x1.X + x2.X

,size,* values *
0,S,11
1,M,2
2,L,50


This can also be used to stack CodedArrays with no overlapping codes

In this example, x1 has data for S and M, while x3 has data for L and XL

In [25]:
x3 = schema.load({'L': 9, 'XL': 15}, 'size')
# This has no overlap
assert len(x1 + x3) == 0
# This will stack the results into a single result
x1.X + x3.X

,size,* values *
0,S,1
1,M,2
2,L,9
3,XL,15


As a more practical example, suppose we wanted to adjust the costs of all the medium widgets by 2x.

Multiplying by an object with only size=M will lose widget cost data for everything which is not size=M.

In [26]:
cost_multiplier = schema.load({'M': 2}, 'size')
widget_cost * cost_multiplier

shape,Triangle,Rectangle,Circle
size,,,
M,22.2,23.6,28


We could manually add a multiplier value of 1 for all other sizes, but that would be verbose and potentially dangerous if we ever add a new widget size in the future and forget to update our multiplier object.

A better solution is to expand our multiplier with a default value of 1 for all unknown sizes.

In [27]:
widget_cost * cost_multiplier.X[1]

shape,Triangle,Rectangle,Circle
size,,,
S,4.5,5.05,7
M,22.2,23.6,28
L,,19.25,25
XL,,,36


### Expanding can create NULL codes

Expanding with matching dimensions or disjoint dimensions will never create a NULL code, but expanding with subset or partially disjoint dimensions might create NULL codes in the results.

Let's look at how this happens.

In [28]:
x

,size,* values *
0,S,1
1,M,2
2,L,3
3,XL,4


In [29]:
z = schema.load([
    ['S', 'Circle', 15.0],
    ['M', 'Triangle', 23.0],
    ['M', 'Circle', 55.2]
], ['size', 'shape'])
z

,size,shape,* values *
0,S,Circle,15.0
1,M,Triangle,23.0
2,M,Circle,55.2


x has dimensions which are a subset of z's dimensions.

Expanding z will create a NULL for the shape because x has no shape dimension for size={'L', 'XL'}.

In [30]:
x + z.X

shape,∅,Triangle,Circle
size,,,
S,,,16
M,,25,57.2
L,3,,
XL,4,,


# Functions

This section demonstrates several useful functions in dimsum and their application

In [31]:
x

,size,* values *
0,S,1
1,M,2
2,L,3
3,XL,4


In [32]:
y

,quality,* values *
0,Bad,-1
1,Okay,1
2,Good,5


In [33]:
w = schema.load([
    [size[s+1], days[d+1], np.random.random()*100] for d in range(180) for s in range(3) if (d + s) % 11 != 0
], ['size', 'days'])
w

,size,days,* values *
0,S,2020-03-16,2.674617
1,S,2020-03-17,8.060203
2,S,2020-03-18,3.926947
3,S,2020-03-19,5.808187
4,S,2020-03-20,16.946137
...,...,...,...
486,L,2020-09-06,89.836862
487,L,2020-09-07,36.866924
488,L,2020-09-08,53.545404
489,L,2020-09-09,51.414193


### CodedArray implements the array interface of numpy

This allowing many of the math operations from numpy to work with CodedArrays.

There is some numba jitting happening, so the first time a new function is used, you might notice a delay. Running the cell
again will be much faster.

In [34]:
np.sin(x)

,size,* values *
0,S,0.841471
1,M,0.909297
2,L,0.141120
3,XL,-0.756802


In [35]:
np.arctan2(widget_cost, x)

shape,Triangle,Rectangle,Circle
size,,,
S,1.35213,1.37531,1.4289
M,1.39253,1.4029,1.4289
L,,1.4162,1.45137
XL,,,1.46014


In [36]:
y << x

size,S,M,L,XL
quality,,,,
Bad,-2,-4,-8,-16
Okay,2,4,8,16
Good,10,20,40,80


### Pivot and Reduce

Aggregating dimensions (ROLLUP or GROUPBY) is done by pivoting the CodedArray, then reducing along either the rows or columns.

The default aggregator is summation, but other are available in the `ds.op` namespace.

In [37]:
pivoted_cost = widget_cost.pivot(left='size')
pivoted_cost

shape,Triangle,Rectangle,Circle
size,,,
S,4.5,5.05,7
M,11.1,11.8,14
L,,19.25,25
XL,,,36


In [38]:
pivoted_cost.reduce_rows()

,size,* values *
0,S,16.55
1,M,36.90
2,L,44.25
3,XL,36.00


In [39]:
pivoted_cost.reduce_columns(ds.op.min)

,shape,* values *
0,Triangle,4.50
1,Rectangle,5.05
2,Circle,7.00


Reduction to scalar is also always available from flat or pivoted form.

In [40]:
widget_cost.reduce(ds.op.max)

36.0

### Where and Filter

`ds.where(condition, true_vals, false_vals)` allows picking items from two objects based on a condition. This is identical to `np.where` and very similar to Excel's `IF`.

In [41]:
ds.where(y < 0, 0, y)

,quality,* values *
0,Bad,0
1,Okay,1
2,Good,5


Filter is just like where but only keeps values based on a condition.

This returns a new object rather than modifying inplace.

In [42]:
x.filter(x > 2)

,size,* values *
0,L,3
1,XL,4


### Filtering and selecting based on codes

`obj.match()` is used for the purpose of filtering or selecting based on the dimension values rather than the actual data in the CodedArray.

In [43]:
widget_cost.match(size='S')

,size,shape,* values *
0,S,Triangle,True
1,S,Rectangle,True
2,S,Circle,True
3,M,Triangle,False
4,M,Rectangle,False
5,M,Circle,False
6,L,Rectangle,False
7,L,Circle,False
8,XL,Circle,False


In [44]:
widget_cost.filter(widget_cost.match(size='S'))

,size,shape,* values *
0,S,Triangle,4.50
1,S,Rectangle,5.05
2,S,Circle,7.00


In [45]:
ds.where(
    widget_cost.match(shape='Circle', size={'M', 'L', 'XL'}),
    3.14159,
    widget_cost
)

,size,shape,* values *
0,S,Triangle,4.50000
1,S,Rectangle,5.05000
2,S,Circle,7.00000
3,M,Triangle,11.10000
4,M,Rectangle,11.80000
5,M,Circle,3.14159
6,L,Rectangle,19.25000
7,L,Circle,3.14159
8,XL,Circle,3.14159


`obj.codes()` is used for more advanced selecting based on the code. It is especially useful for selecting ranges.

In [46]:
w.filter((w.codes('days') >= schema.encode(days='2020-06-01')) & (w.codes('days') < schema.encode(days='2020-06-15'))).pivot(left='size')

days,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14
size,,,,,,,,,,,,,,
S,0.86608,13.8281,23.4422,16.0999,88.7752,47.4437,87.7566,37.4513,95.2777,63.8065,,6.31425,6.20016,51.8807
M,41.654,19.4974,95.1874,62.1159,33.7447,57.7972,9.85465,29.3209,27.4376,,50.7352,47.6674,27.1142,14.4919
L,53.9062,39.2724,55.0067,48.2666,20.7113,96.4711,46.4676,85.4324,,29.9988,40.0652,21.0924,75.4425,33.278


To get a single value out of a CodedArray, filter using a fully specified match, then reduce.

In [47]:
w.filter(w.match(size='L', days='2020-06-12')).reduce()

21.09236215225645

### Calendar Builtins

When a schema has CalendarDimensions, many calendar datasets are automatically built.

These can be used for filtering or to convert between different frequencies.

In [48]:
list(schema.calendar)

['days.day',
 'days.month',
 'days.quarter',
 'days.year',
 'days.days_in_month',
 'days.days_in_year',
 'days.days_in_quarter',
 'quarters.quarter',
 'quarters.year',
 'quarters.days_in_year',
 'quarters.days_in_quarter',
 'days->quarters']

In [49]:
schema.calendar['quarters.days_in_quarter']

,quarters,* values *
0,2020Q1,91
1,2020Q2,91
2,2020Q3,92
3,2020Q4,92


Find the value of w on the last day of each month

In [50]:
w.filter(schema.calendar['days.day'] == schema.calendar['days.days_in_month'])

size,S,M,L
days,,,
2020-03-31,63.1373,46.6195,1.42871
2020-04-30,10.5733,64.3686,54.3464
2020-05-31,,38.6769,85.9188
2020-06-30,5.24076,17.5723,7.50687
2020-07-31,52.9564,82.0874,99.8335
2020-08-31,64.8246,98.9871,81.0204


Let's aggregate w into quarterly data by summing daily data.

The days->quarters calendar data has a 1 where a day and quarter overlap. Using this, we can multiply w by days->quarters, then reduce along the columns to remove the days.

In [51]:
d2q = schema.calendar['days->quarters']
d2q

quarters,2020Q1,2020Q2,2020Q3
days,,,
2020-03-15,1,,
2020-03-16,1,,
2020-03-17,1,,
2020-03-18,1,,
2020-03-19,1,,
...,...,...,...
2020-09-06,,,1
2020-09-07,,,1
2020-09-08,,,1


Multiplying by d2q doesn't add any data, but simply tags each value with its corresponding Quarter. This happens because of the way d2q is set up with each Day having exactly one pairing to a Quarter.

In [52]:
w * d2q

size,S,S,S,M,M,M,L,L,L
quarters,2020Q1,2020Q2,2020Q3,2020Q1,2020Q2,2020Q3,2020Q1,2020Q2,2020Q3
days,,,,,,,,,
2020-03-15,,,,48.5389,,,92.315,,
2020-03-16,2.67462,,,81.8318,,,65.0004,,
2020-03-17,8.0602,,,34.5642,,,97.0811,,
2020-03-18,3.92695,,,5.07678,,,42.5964,,
2020-03-19,5.80819,,,87.1123,,,87.516,,
...,...,...,...,...,...,...,...,...,...
2020-09-06,,,10.0074,,,,,,89.8369
2020-09-07,,,,,,57.5864,,,36.8669


Now that the data has both monthly and quarterly labels, we can reduce along the columns to eliminate the months and aggregate into quarterly data.

In [53]:
(w * d2q).reduce_columns()

,size,quarters,* values *
0,S,2020Q1,457.303104
1,S,2020Q2,4198.265676
2,S,2020Q3,3484.730601
3,M,2020Q1,741.440265
4,M,2020Q2,4214.060305
5,M,2020Q3,3361.400477
6,L,2020Q1,753.569918
7,L,2020Q2,3737.294603
8,L,2020Q3,3677.105848


### Shifting Codes

Calendar Dimensions are a common place for wanting to shift values along the dimension (i.e. lag the values by 1 month).

Dimsum has a `shift` method which does this for a single dimension at a time. The shift parameter can be a scalar or a CodedArray which shifts each element independently. Because arbitrary shifting might causes multiple values to collide, an aggregation must be provided.

Anything which is shifted beyond the allowable values for a dimension will be assigned the NULL value for that dimension.

In [54]:
stock_price = schema.load([
    ['AAPL', '2020-06-01', 79.92], ['AAPL', '2020-06-02', 80.29], ['AAPL', '2020-06-03', 80.74],
    ['MSFT', '2020-06-01', 181.49], ['MSFT', '2020-06-02', 183.56], ['MSFT', '2020-06-03', 184.01]
], ['ticker', 'days']).pivot(top='days')

Shifting the stock price by 1 day will make 2020-06-01 become 2020-06-02, etc

In [55]:
stock_price.shift('days', 1)

days,2020-06-02,2020-06-03,2020-06-04
ticker,,,
AAPL,79.92,80.29,80.74
MSFT,181.49,183.56,184.01


Our "days" dimension only has 180 days, from 2020-03-15 to 2020-09-10.

Shifting by 101 days will shift two of our values beyond what the dimension can represent, and they will accumulate in the NULL dimension label.

In [56]:
stock_price.shift('days', 101)

days,∅,2020-09-10
ticker,,
AAPL,161.03,79.92
MSFT,367.57,181.49


Adding stock prices doesn't make much sense, so let's calculate the max instead

In [57]:
stock_price.shift('days', 101, agg=ds.op.max)

days,∅,2020-09-10
ticker,,
AAPL,80.74,79.92
MSFT,184.01,181.49


Suppose we want to shift AAPL by 1 day, but MSFT by -2 days

In [58]:
uneven_shifter = schema.load({'AAPL': 1, 'MSFT': -1}, 'ticker')
stock_price.shift('days', uneven_shifter)

days,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04
ticker,,,,,
AAPL,,,79.92,80.29,80.74
MSFT,181.49,183.56,184.01,,


Suppose we want to shift widget sizes up, but avoid pushing XL into the NULL label.

We can achieve that easily with a rule for XL to shift by 0. To make everything else shift by 1, we can use Expansion syntax with a default fill value of 1.

In [59]:
size_shifter = schema.load({'XL': 0}, 'size').X[1]

In [60]:
widget_cost.shift('size', size_shifter)

,size,shape,* values *
0,M,Triangle,4.50
1,M,Rectangle,5.05
2,M,Circle,7.00
3,L,Triangle,11.10
4,L,Rectangle,11.80
5,L,Circle,14.00
6,XL,Rectangle,19.25
7,XL,Circle,61.00


If the logic is hard to follow, we can manually align those to more clearly see what is actually going on.

You can see that size_shifter was expanded to the same size and elements as widget_cost. Anything not specified in size_shifter gets a default value of 1, while the explicit XL=0 remains, avoiding XL shifting into the NULL label.

In [61]:
size_shifter.align(widget_cost)

shape,Triangle,Rectangle,Circle
size,,,
S,1,1,1
M,1,1,1
L,,1,1
XL,,,0


### Example: business day logic

Another example shows how shifting can achieve things like business day logic.

Suppose orders come in from the internet on every day, but orders are only processed on days the business is open. We want to shift those orders to the next available business day to reflect this.

Rather than writing a for loop with a nested while loop to calculate the next business day for any given day (which might be 1 or 2 for a weekend, but could be longer for 3-day weekends or holidays).

The easier solution is to manually build a shifter which indicates for every day how far the next business day is, then perform the shift.

In [62]:
# Only write the non-zero shifts
business_day_shifter = schema.load({
    '2020-07-03': 3,  # 4th of July observed
    '2020-07-04': 2,  # Saturday
    '2020-07-05': 1,  # Sunday
    '2020-07-11': 2,  # ...
    '2020-07-12': 1,
    '2020-07-18': 2,
    '2020-07-19': 1,
    '2020-07-25': 2,
    '2020-07-26': 1,
}, 'days').X  # Expand with default 0 here

Dimsum has ds.ones_like, ds.zeros_like, and ds.full_like to build constant-valued arrays, similar to the numpy variants of the same names.

Use ds.full_like with range filtering to build up an object with 10 orders per day for each day in July 2020

In [63]:
all_days = schema.dimension_enums('days')
july_days = all_days.filter((all_days.codes() >= schema.encode(days='2020-07-01')) & (all_days.codes() <= schema.encode(days='2020-07-31')))
july_orders = ds.full_like(july_days, 10)
july_orders

,days,* values *
0,2020-07-01,10
1,2020-07-02,10
2,2020-07-03,10
3,2020-07-04,10
4,2020-07-05,10
5,2020-07-06,10
6,2020-07-07,10
7,2020-07-08,10
8,2020-07-09,10
9,2020-07-10,10


Now apply the shift to move weekend and holiday orders to business days, summing the totals.

In [64]:
july_orders.shift('days', business_day_shifter)

,days,* values *
0,2020-07-01,10
1,2020-07-02,10
2,2020-07-06,40
3,2020-07-07,10
4,2020-07-08,10
5,2020-07-09,10
6,2020-07-10,10
7,2020-07-13,30
8,2020-07-14,10
9,2020-07-15,10


### Constructors

CodedArrays are built from a schema using one of four input types:
1. pd.Series
2. pd.DataFrame
3. dict
4. list of lists

### Series Constructor

A CodedArray is most similar to a pandas Series, although it is more restricted because it requires the index to be unique.

Assuming the Series has a MultiIndex with labeled levels, the schema can load the Series without any additional input.

In [65]:
s = pd.Series([42, 39, 14], index=pd.Index(['Circle', 'Triangle', 'Rectangle'], name='shape'))

In [66]:
s2 = schema.load(s)
s2

,shape,* values *
0,Triangle,39
1,Rectangle,14
2,Circle,42


### DataFrame Constructor

A DataFrame may also be used. It is assumed that the dimension data is in columns, not the index. Therefore a list of dimension columns must be specified along with the DataFrame.

If there is exactly one more column than the indicated dimension columns, it will be assumed to contain the data values. Otherwise, the value column must be specified.

In [67]:
df = pd.DataFrame([['Triangle', 42], ['Circle', 39], ['Rectangle', 14]], columns=['shape', 'value'])

In [68]:
schema.load(df, ['shape'])

,shape,* values *
0,Triangle,42
1,Rectangle,14
2,Circle,39


### Dictionary Constructor

A Python dict may be used with each key containing the dimension labels and the value being the data. Loading requires a list of dimension names in the same order as the keys.

If the dict has a single dimension, a string may be used. If more than one dimension exists, the keys must be tuples.



In [69]:
d1 = {('S', 'Circle'): 17.1, ('M', 'Triangle'): 33.8, ('L', 'Circle'): 11.7}

In [70]:
schema.load(d1, ['size', 'shape'])

,size,shape,* values *
0,S,Circle,17.1
1,M,Triangle,33.8
2,L,Circle,11.7


In [71]:
schema.load({'S': -1, 'M': 0, 'L': 1}, 'size')

,size,* values *
0,S,-1
1,M,0
2,L,1


### Lists of Lists Constructor

A List of Lists represents each row in the resulting table. Each row must have a length one greater than the number of dimensions. When loading, the list of dimensions must be provided.

In [72]:
ll = [['Circle', 'S', 17.1], ['Triangle', 'M', 33.8], ['Circle', 'L', 11.7]]

In [73]:
schema.load(ll, ['shape', 'size'])

,size,shape,* values *
0,S,Circle,17.1
1,M,Triangle,33.8
2,L,Circle,11.7


# Advanced Alignment

When the alignment of results doesn't make sense, there are ways to manually align and see what is going on prior to doing the actual calculation.

`obj.align(other_obj)` will show obj post-alignment. This takes care of any expansion (ex. `obj.X.align(other_obj)`).

In [74]:
x

,size,* values *
0,S,1
1,M,2
2,L,3
3,XL,4


In [75]:
z

,size,shape,* values *
0,S,Circle,15.0
1,M,Triangle,23.0
2,M,Circle,55.2


In [76]:
x.align(z)

shape,Triangle,Circle
size,,
S,,1
M,2,2


In [77]:
x.align(z.X)

shape,∅,Triangle,Circle
size,,,
S,,,1
M,,2,2
L,3,,
XL,4,,


If we want to look at z post-alignment, we could write `z.align(x)`.

But if the goal is to look at both objects post-alignment, use `ds.align(obj1, obj2)`.

In [78]:
x_align, z_align = ds.align(x, z.X)

In [79]:
x_align

shape,∅,Triangle,Circle
size,,,
S,,,1
M,,2,2
L,3,,
XL,4,,


In [80]:
z_align

shape,∅,Triangle,Circle
size,,,
S,,,15
M,,23,55.2
L,0,,
XL,0,,


### Expanding using the values of another object

Typically, expansion uses a fixed scalar value to fill in missing values that another object has.

In [81]:
z.X[99].align(x)

shape,∅,Triangle,Circle
size,,,
S,,,15
M,,23,55.2
L,99,,
XL,99,,


Expansion can also take another object as the fill value, in which case it immediately computes the result, filling in any missing values with values from the other object.

In [82]:
z.X[x]

shape,∅,Triangle,Circle
size,,,
S,,,15
M,,23,55.2
L,3,,
XL,4,,


### Cross Alignment

Expansion only works for matching dimensions where one object has a label not contained in the other.

For subset and partially disjoint alignment, this can result in NULL labels for the expanded values (as seen above for L and XL).

Rather than giving L=3 and XL=4 a NULL shape, we want the value to show up in both Triangle and Circle (ex. L Triangle=3, L Circle=3). We also want S=1 to fill in the Triangle value. In essence, `x` gives a value for each of the sizes and we want to fill those size values for all known shapes in `z`.

This is called a **cross alignment**.

In [83]:
x.cross_align(z)

shape,Triangle,Circle
size,,
S,1,1
M,2,2
L,3,3
XL,4,4


Comparing the result of normal expansion versus cross alignment is helpful in seeing the difference

In [84]:
normal_expansion_add = x + z.X
normal_expansion_add

shape,∅,Triangle,Circle
size,,,
S,,,16
M,,25,57.2
L,3,,
XL,4,,


In [85]:
cross_align_add = x.cross_align(z) + z.X
cross_align_add.pivot(left='size')

shape,Triangle,Circle
size,,
S,1,16
M,25,57.2
L,3,3
XL,4,4


# Under the Hood

Dimsum is built using [SuiteSparse:GraphBLAS](https://people.engr.tamu.edu/davis/GraphBLAS.html), a C library for sparse linear algebra written by Tim Davis.

A key feature of SuiteSparse:GraphBLAS is the hypersparse (also known as double-compressed) format, which makes storage of indices up to 2**60 very efficient.

In [86]:
x = schema.load(
    [['S', 'Triangle', 4.50],
     ['S', 'Circle', 7.00],
     ['M', 'Rectangle', 11.80],
     ['L', 'Rectangle', 19.25],
     ['L', 'Circle', 25.00],
     ['XL', 'Triangle', 36.00],
    ], ['size', 'shape']
)
x

,size,shape,* values *
0,S,Triangle,4.50
1,S,Circle,7.00
2,M,Rectangle,11.80
3,L,Rectangle,19.25
4,L,Circle,25.00
5,XL,Triangle,36.00


To look at the raw object requires digging into [grblas](https://github.com/metagraph-dev/grblas), a Python wrapper around SuiteSparse:GraphBLAS.

We will convert the raw object to COO format for viewing.

In [87]:
x.obj.vector.to_values()

(array([163840, 229376, 327680, 458752, 491520, 557056], dtype=uint64),
 array([ 4.5 ,  7.  , 11.8 , 19.25, 25.  , 36.  ]))

We see that `x` is stored with an index of codes and values. The meaning of the codes is given by the schema, which the underlying Vector knows nothing about.

Getting the codes in Dimsum is simply copying the index into the values.

In [88]:
x.codes()

,size,shape,* values *
0,S,Triangle,163840
1,S,Circle,229376
2,M,Rectangle,327680
3,L,Rectangle,458752
4,L,Circle,491520
5,XL,Triangle,557056


In [89]:
x.codes().obj.vector.to_values()

(array([163840, 229376, 327680, 458752, 491520, 557056], dtype=uint64),
 array([163840, 229376, 327680, 458752, 491520, 557056]))

Pivoted data is stored as a Matrix rather than a Vector.

While a Matrix in theory could hold another 2**60 codes in the index for the 2nd dimension, the bits never overlap. A dimension on the left is never utiltized on the top, meaning the index of the Vector is simply the sum of the left and top index in the Matrix.

In [90]:
matrix_raw_data = x.pivot(left='size').obj.matrix.to_values()
matrix_raw_data

(array([131072, 131072, 262144, 393216, 393216, 524288], dtype=uint64),
 array([32768, 98304, 65536, 65536, 98304, 32768], dtype=uint64),
 array([ 4.5 ,  7.  , 11.8 , 19.25, 25.  , 36.  ]))

In [91]:
matrix_raw_data[0] + matrix_raw_data[1]

array([163840, 229376, 327680, 458752, 491520, 557056], dtype=uint64)